In [1]:
###################################
# FUNCTION TO WRITE UNIQUE VALUES #
###################################

# Function to count frquency of values in the rows of a dataset
# INPUT:
# df_temp [pyspark-dataframe] Dataframe on which operation needs to be performed
# rowname [string] name of the string whose values need to counted
# filename [string] name of the file which will store the result
# OUTPUT:
# CSV file with results of the operation
def unique_writer(df_temp, rowname, filename):
    df_unique = df_temp.groupby(rowname).count().sort(col("count").desc())
    df_unique.toPandas().to_csv(filename, index=False)

In [2]:
#####################################################
# FUNCTION TO COUNT UNWANTED VALUES IN EVERY COLUMN #
#####################################################

# INPUT :
# dataframe_name [string] name of the the dataframe on which operation needs to be preformed
# file_name [string] name of the the file in which results need to be stored in csv format
# OUTPUT:
# CSV file with output of the operation
def count_unwanted(dataframe_address, file_name):

    df_temp = spark.read.csv(dataframe_address, header=True)

    # unique_count = []
    nan_count = []
    null_count = []
    q_count = []
    zero_count = []
    column_index = []

    for i in df_temp.columns:
        column_index.append(i)
        # unique_count.append(df_temp.select(i).distinct().count())
        nan_count.append(df_temp.where(df_temp[i].isNull()).count())
        null_count.append(df_temp.filter(df_temp[i] == 'NaN').count())
        q_count.append(df_temp.filter(df_temp[i] == '?').count())
        zero_count.append(df_temp.filter(df_temp[i] == 0).count())

    # unwanted_data = {"unique":unique_count, "nan_count": nan_count, "null_count": null_count, "q_count": q_count, "zero_count": zero_count}
    unwanted_data = {"nan_count": nan_count, "null_count": null_count, "q_count": q_count, "zero_count": zero_count}

    pd.DataFrame(unwanted_data, index=column_index).to_csv(file_name)

# MAIN BODY OF THE PROGRAM

In [3]:
# Importing Libraries and functionality
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct
import pandas as pd

spark = SparkSession.builder.getOrCreate() # Creating Spark Session

## 1) Exploratory Data Analysis

### 1.1) Understanding Enteries

#### 1.1.1) allergies.csv

In [4]:
df_temp = spark.read.csv("dataset_project_1/allergies.csv", header=True)

unique_writer(df_temp, "DESCRIPTION", "eda/understanding_entries/allergies_uvc1.csv") # Unique Values in the the DESCRIPTION column
unique_writer(df_temp, "PATIENT", "eda/understanding_entries/allergies_uvc2.csv") # Unique Values in the patient column

# Determining if there are patients with multiple ongoing allergies
df_temp = df_temp.where(df_temp["STOP"].isNull())
unique_writer(df_temp, "PATIENT", "eda/understanding_entries/allergies_nuvc1.csv")

# Unwanted Data
count_unwanted("dataset_project_1/allergies.csv", "eda/understanding_entries/allergies_unwanted.csv")

| **File**                 | **Operation**                                                                     |
|:-------------------------|:----------------------------------------------------------------------------------|
| allergies_uvc1.csv       | Unique Values Count on "DESCRIPTION"                                              |
| allergies_uvc2.csv       | Unique Values Count on "PATIENT"                                                  |
| allergies_nuvc1.csv      | Unique Values Count on "PATIENT" on dataframe filtered by null   values in "STOP" |
| allergies_unwanted.csv   | Count of unwanted data in the dataframe                                           |


• **Allergy to mould** is the most frequent entry and Allergy to soya is the least common entry (see
<i>allergies_uvc1.csv</i>)
<br> <br>
• It is possible for a single patient id to be listed against multiple enteries in allergies.csv, (see <i>allergies uvc2.csv</i>) however is it possible for a patient to have multiple allergies remain to be seen. 
<br> <br>
• It is possible for a patient to have multiple ongoing allergies (see <i>allergies_nuvc1.csv</i> )
<br> <br>
• "STOP" column in allergies.csv have some null values, but this cannot be attributed to unwanted or
missing data, it represents entries with ongoing allergies.

#### 1.1.2) careplans.csv

In [5]:
# CAREPLANS #

df_temp = spark.read.csv("dataset_project_1/careplans.csv", header=True)

# Unique Values Count
unique_writer(df_temp, "PATIENT", "eda/understanding_entries/careplans_uvc1.csv") # PATIENT Column
unique_writer(df_temp, "DESCRIPTION", "eda/understanding_entries/careplans_uvc2.csv") # DESCRIPTION column
unique_writer(df_temp, "REASONDESCRIPTION", "eda/understanding_entries/careplans_uvc3.csv") # REASONDESCRIPTION column

# Unique Values Count on Datafarem filtered by NaN values
df_temp1 = df_temp.where(df_temp["STOP"].isNull()) # Filtering the data by null values in STOP column
unique_writer(df_temp1, "PATIENT", "eda/understanding_entries/careplans_nuvc1.csv") # unique values count on PATIENT column

df_temp2 = df_temp.where(df_temp["REASONDESCRIPTION"].isNull()) # Filtering the data by NaN values in REASONDESCRIPTION column
unique_writer(df_temp2, "DESCRIPTION", "eda/understanding_entries/careplans_nuvc2.csv") # Unique values on the DESCRITION column


count_unwanted("dataset_project_1/careplans.csv", "eda/understanding_entries/careplans_unwanted.csv") # Unwanted Data

| **File**            | Operation**                                                                                         |
|:-----------------------|:-------------------------------------------------------------------------------------------------|
| careplans_uvc1.csv     | Unique Values Count on "PATIENT"                                                                 |
| careplans_uvc2.csv     | Unique Values Count on "DESCRIPTION"                                                             |
| careplans_uvc3.csv     | Unique Values Count on "REASONDESCRIPTION"                                                       |
| careplans_nuvc1.csv    | Unique Values Count on "PATIENT" on dataframe filtered by null values in "STOP"                  |
| careplans_nuvc2.csv    | Unique Values Count on "DESCRIPTION" on dataframe filtered by null values in "REASONDESCRIPTION" |
| careplans_unwanted.csv | Count of unwanted data in the dataframe                                                          |

• It is possible for a patient to have multiple careplans (see <i>careplans_uvc1.csv</i>), however further investigation is required to check if it is possible for a patient to have multiple ongoing careplans.
<br> <br>
• To study the most common reasons careplans are used for, it was deterimined to perform a unique Values
count on the columns named "DESCRIPTION" and "REASONDESCRIPTION" of the careplans.csv. 
<br> <br>
Perfomring a unique values count on either one of the column will not be sufficientcient as single value in the
"DESCRIPTION" colummn could be listed against multiple entries in the "REASONDESCRIPTION"
column. 
<br> <br>
• Careplans are most commonly used for **respiratory therapy** (see <i>careplans_uvc2.csv</i>) 
<br> <br>
• Careplans are most commonly used for treatment of **Acute Bronchitis(Disorder)** (see <i>careplans_uvc3.csv</i>)
which is a respiratory disorder thus justifying the fact that Respiratory Therepy is the most commonly
occuring value in the "DESCRIPTION" column. 
<br> <br>
The "STOP" column contains NaN values but it could be attributed to ongoing careplans and cannot
be considered as unwanted or bad data. The "REASONDECSRIPTION" column also contains NaN
values it represents the use of careplans for non-medical reasons. Thus we can say that careplans.csv
does not contains any unwanted data (see <i>careplans_unwanted.csv</i>). 
<br> <br>
• For non-medical reason (i.e. when "REASONDESCRIPTION" has a **NaN value**) the careplans are most
commonly used for **Self-care interventions (procedure)** (see <i>careplans_nuvc2.csv</i>).

#### 1.1.3) conditions.csv

In [6]:
# CONDITIONS #

df_temp = spark.read.csv("dataset_project_1/conditions.csv", header=True) # Reading the dataframe

# Unique Values Count
unique_writer(df_temp, "DESCRIPTION", "eda/understanding_entries/conditions_uvc1.csv") # DESCRIPTION Column

# Unique Values Count on null filtered Dataframe
df_temp1 = df_temp.where(df_temp["STOP"].isNull()) # Filtering by NaN values in STOP ccolumn
unique_writer(df_temp1, "DESCRIPTION", "eda/understanding_entries/conditions_nuvc1.csv") # performing unique values count - The conditions in this file will be called "Assumed chronic"

# Dtermining if any of the conditions which do not have a stop date has stop dates in any of the enteries
chronic_conditions = set(df_temp1.toPandas()["DESCRIPTION"].tolist()) # Set of all the conditions which do not have a STOP date
df_temp2 = df_temp.where(df_temp["DESCRIPTION"].isin(chronic_conditions)) # Filtering the dataframe with the above set
df_temp3 = df_temp2.where(df_temp2["STOP"].isNotNull()) # Filtering the dataset again to have non Null Values
unique_writer(df_temp3, "DESCRIPTION", "eda/understanding_entries/conditions_psudo_chroninc.csv") # Writing the results to a file

# NOTE: Psudo Chronic Conditions is self coined term which represents conditions which were classsified as chroninic(i.e.
# having no stop date ) in some enteries but actually have STOP dates in some other enteries.

# Making a list of chronic situations
df_temp1 = spark.read.csv("eda/understanding_entries/conditions_nuvc1.csv", header=True) # File with conditions from enteris which have no STOP date
df_temp2 = spark.read.csv("eda/understanding_entries/conditions_psudo_chroninc.csv", header=True) # File with enteries with psudo chronic conditions
assumed_chronic = set(df_temp1.toPandas()["DESCRIPTION"].tolist()) # set of conditions which have no STOP dates
psudo_chronic = set(df_temp2.toPandas()["DESCRIPTION"].tolist()) # set of psudo chronic conditions
real_chronic = assumed_chronic - psudo_chronic # chronic conditions
df_chronic = pd.Series(list(chronic_conditions)) # Crearing a dataframe of chronic conditions
df_chronic.to_csv("preprocessed/conditions_chronic.csv", index=False, header=False) # writing chronic conditions to a file

| **File**                     | **Operation**                                                                                                                         |
|:-----------------------------|:--------------------------------------------------------------------------------------------------------------------------------------|
| conditions_uvc1.csv          | Unique Values Count on "DESCRIPTION"                                                                                                  |
| conditions_nuvc1.csv         | Unique Values Count on "DESCRIPTION"  on dataframe filtered by null values in "STOP"                                                  |
| conditions_psudo_chronic.csv | List of psudo chronic conditions i.e. conditions which have no "STOP" values in some cases but do have a "STOP" value somewhere else. |

• **Viral Sinusitis (disorder)** was the most common condition (see <i>conditions_uvc1.csv</i>)
<br> <br>
• It was observed that the "STOP" column in the <i>careplans.csv</i> had some **NaN values**, thus giving the impression that certain conditions could be chronic in nature thus the dataset was filtered by **NaN values** in the "STOP" column and unique values count was performed on the "DESCRIPTION" column, the results of this operation are stored in <i>condition_nuvc1.csv</i>. However this approach has a major drawback,
some conditions which do not have a stop date in certain cases do have stop dates in some other cases.
<br> <br>
• In order to check weather a condition is trully chronic in nature or not, further investigation was required. The dataset was checked to see if any of the conditions in the <i>conditions_nuvc1.csv </i> has stop
dates anywhere in the dataset. The enteries pertaining to this investigation are recoded in <i>conditions_psudo_chronic.csv</i>. 
<br> <br>
• To get the list of all chronic conditions, a difference operation was performed between the list of conditions in <i>conditions_nuvc1.csv </i> and conditions psudo chronic.csv. The resultant conditions were recorded in <i>conditions_chronic.csv</i> stored in the preprocessed folder.

#### 1.1.4) encounters.csv

In [7]:
# ENCOUNTER #

df_temp = spark.read.csv("dataset_project_1/encounters.csv", header=True)

# Unique Values Count
unique_writer(df_temp, "REASONDESCRIPTION", "eda/understanding_entries/encounters_uvc1.csv") # REASONDESCRIPTION column
unique_writer(df_temp, "DESCRIPTION", "eda/understanding_entries/encounters_uvc2.csv") # DESCRIPTION column
unique_writer(df_temp, "ENCOUNTERCLASS", "eda/understanding_entries/encounters_uvc3.csv") #  ENCOUNTERCLASS column

count_unwanted("dataset_project_1/encounters.csv", "eda/understanding_entries/encounters_unwanted.csv") # Unwanted Data

# Unique Values Count on Dataframe filtered by NaN values
df_temp1 = df_temp.where(df_temp["REASONDESCRIPTION"].isNull()) # Filtering the dataset by NaN values in REASONDECRIPTION column
unique_writer(df_temp1, "DESCRIPTION", "eda/understanding_entries/encounters_nuvc1.csv") # Unique Value Count on DESCRIPTION column
unique_writer(df_temp1, "ENCOUNTERCLASS", "eda/understanding_entries/encounters_nuvc2.csv") # Unique Value Count on ENCOUNTERCLASS column

# Cross Tabulation operations
df_temp.crosstab("REASONDESCRIPTION", "ENCOUNTERCLASS").toPandas().to_csv("eda/understanding_entries/encounters_ct1.csv", index=False) # REASONDESCRIPTION and ENCOUNTERCLASS
df_temp.crosstab("DESCRIPTION", "ENCOUNTERCLASS").toPandas().to_csv("eda/understanding_entries/encounters_ct2.csv", index=False) # DESCRIPTION and ENCOUNTERCLASS
df_temp.crosstab("DESCRIPTION", "REASONDESCRIPTION").toPandas().to_csv("eda/understanding_entries/encounters_ct3.csv", index=False) # DESCRIPTION and REASONDECSRIPTION

# Cross Tabulation operation on Dataframe filtered by NaN operations
# df_temp1 = df_temp.where(df_temp["REASONDESCRIPTION"].isNull())
# df_temp1.crosstab("DESCRIPTION", "ENCOUNTERCLASS").toPandas().to_csv("eda/encounters_REASONDESCRIPTION_null_crosstab.csv", index=False)

| **File**                 | **Operation**                                                                      |
|:-------------------------|:-----------------------------------------------------------------------------------|
| encounters_uvc1.csv      | Unique Values Count on "REASONDESCRIPTION"                                         |
| encounters_uvc2.csv      | Unique Values Count on "DESCRIPTION"                                               |
| encounters_uvc3.csv      | Unique Values Count on "ENCOUNTERCLASS"                                            |
| encounters_nuvc1.csv     | Unique Values Count on "DESCRIPTION" filtered by NaN values "REASONDESCRIPTION"    |
| encounters_nuvc2.csv     | Unique Values Count on "ENCOUNTERCLASS" filtered by NaN values "REASONDESCRIPTION" |
| encounters_ct1.csv       | Cross Tabulation operation between "REASONDESCRIPTION" and "ENCOUNTERCLASS"        |
| encounters_ct2.csv       | Cross Tabulation operation between "DESCRIPTION" and "ENCOUNTERCLASS"              |
| encounters_ct3.csv       | Cross Tabulation between "DESCRIPTION" and "REASONDESCRIPTION" column              |
| encounters_unwanted. csv | Count of Unwanted data on the dataframe                                            |


• Most encounters happen because of non-medical reasons i.e. **NaN values** and the second most common
cause of encounters is **Normal Pregenancy** (see <i>encounters_uvc1.csv</i>).
<br> <br>
• As discussd above the most encounters are due to non-medical reasons, a further analysis reveals that
most encounters happen can be classided as **Well child visit (procedure)** (see <i>encounters_uvc2.csv</i>).
<br><br>
• For most of the encounters the patient was in an ambulatory state i.e. the patient is not bed ridden
and can walk (see <i>encounters_uvc3.csv</i>).
<br> <br>
• <i>encounters.csv</i> was also analyzed for unwanted data and the output was stored in <i>encounters_unwanted.csv</i>. It was observed that only "REASONDESCRIPTION" had **NaN values** and "PAYER COVERAGE"
had some zeros. It is possible that a patient had no insurance coverage at all hence the zeros in the "PAYER COVERAGE" column cannot be treated as unwanted values values however the null values in "REASONDESCRIPTION" column need to be investigated further. Thus at this point it can't be stated with absolute certainty that <i>encounters.csv</i> does not have any unwanted data (see <i>encounters_unwanted.csv</i>).
<br> <br>
• For encounters which are not because any major medical purpouse the most common enounters are for **Well child visit (procedure)** (see <i>encounters_nuvc2.csv</i>) . After comparing the data in <i>encounters_uvc2.csv</i> and <i>encounters_nuvc1.csv</i> we can say that all the encounters of the type **Well child visit(procedure)** are for non-medical reasons.
<br><br>
• It was also observed that most of the encounters for non-medical reason are most commonly of the type **wellness** (see <i>encounters_nuvc2.csv</i>)
<br><br>
• On a closer observation it was noticed that a single entry in "DESCRIPTION" column could be listed against multiple entries in "REASONDESCRIPTION" and "ENCOUNTERCLASS". Thus several cross tabulation operations were performed to understand the distribution of the data across multiple categories.
<br> <br>
• Most of the entries in "REASONDESCRIPTION" and "DESCRIPTION" could be listed across multiple entries of "ENCOULNTERCLASS" (see <i>encounters_ct1.csv</i> and <i>encounters ct2.csv</i> ).
<br> <br>
• It must also be noted that several entries in the "DESCRIPTION" column could be listed against more than one entry in the "REASONDESCRIPTION" column (see <i>encounters_ct3.csv</i>) .
<br> <br>
• The "ENCOUTERCLASS" column has several unique values, we are particularly interested in the
encounters where "ENCOUTERCLASS" is **emergency** and **urgentcare**.

#### 1.1.5) imaging_studies.csv

In [8]:
# IMAGING STUDIES #

df_temp = spark.read.csv("dataset_project_1/imaging_studies.csv", header=True)

# Unique Values Count
unique_writer(df_temp, "BODYSITE_DESCRIPTION", "eda/understanding_entries/imaging_studies_uvc1.csv") # BODYSITE_DESCRIPTION
# unique_writer(df_temp, "MODALITY_DESCRIPTION", "eda/imaging_studies_MODALITY_DESCRIPTION.csv") # MODATLITY_DESCRIPTION
unique_writer(df_temp, "SOP_DESCRIPTION", "eda/understanding_entries/imaging_studies_uvc2.csv") # SOP_DESCRIPTION

count_unwanted("dataset_project_1/imaging_studies.csv", "eda/understanding_entries/imaging_studies_unwanted.csv")

# Cross Tabulation Operation
# df_temp.crosstab("BODYSITE_DESCRIPTION", "MODALITY_DESCRIPTION").toPandas().to_csv("eda/imaging_studies_crosstab_BODYSITE_DESCRIPTION_MODALITY_DESCRIPTION.csv", index=False)
df_temp.crosstab("BODYSITE_DESCRIPTION", "SOP_DESCRIPTION").toPandas().to_csv("eda/understanding_entries/imaging_studies_ct1.csv", index=False) # BODYSITE_DESCRIPTION and SOP_DESCRIPTION
df_temp.crosstab("MODALITY_DESCRIPTION", "SOP_DESCRIPTION").toPandas().to_csv("eda/understanding_entries/imaging_studies_ct2.csv", index=False) # MODALITY_DESCRIPTION and SOP_DESCRIPTION

| **File**                     | **Operation**                                                              |
|:-----------------------------|:---------------------------------------------------------------------------|
| imaging_studies_uvc1.csv     | Unique Values Count on "BODYSTIE_DECSRIPTION"                              |
| imaging_studies_uvc2.csv     | Unique Values Count on "MODALITY_DESCRIPTION"                              |
| imaging_studies_ct1.csv      | Cross Tabulation operation on "BODYSITE_DESCRIPTION" and "SOP_DESCRIPTION" |
| imaging_studies_ct2.csv      | Cross Tabulation operation on "MODALITY_DESCRIPTION" and "SOP_DESCRIPTION" |
| imaging_studies_unwanted.csv | Count of Unwanted Data in Dataframe                                        |

• **Thoracic Structutre (Body Study)** is the most examined body structure (see <i>imaging_studies_uvc1.csv</i>)
<br><br>
• **Digital X-ray** is the most commonly performed procedure (see imaging <i>studies_uvc2.csv</i>).
<br><br>
• It can be noted that multiple tests could be performed on a single body part.
<br><br>
• **Thoracic structure (body structure)** is the only value in "BODYSITE_DECSRIPTION" that is listed against multiple values in the "SOP DESCRIPTION" column (see <i>imaging_studies_ct1.csv</i>). It must also be noted that "BODYSITE_DESCRIPTION" also have values **Thoracic structure** and **thoracic** which are listed against a single value in "SOP_DESCRIPTION" column. However if these represent different body site or are just a different values for the the same body site is a topic of further analysis. All ohter values in "BODYSITE_DESCRIPTION" column are listed against a single value in "SOP_DESCRIPTION" column.
<br><br>
• <i>imaging_studies.csv</i> does not contains any unwanted data (see <i>imaging_studies_unwanted.csv</i>).

#### 1.1.6) immunizations.csv

In [9]:
# IMMUNIZATION #

df_temp = spark.read.csv("dataset_project_1/immunizations.csv", header=True)

# Unique Values Count
unique_writer(df_temp, "DESCRIPTION", "eda/understanding_entries/imunization_uvc1.csv")

count_unwanted("dataset_project_1/immunizations.csv", "eda/understanding_entries/immunization_unwanted.csv") # Unwanted Data

| File                       | Operation                                   |
|:---------------------------|:--------------------------------------------|
| immunization_uvc1.csv      | Unique Values Count on "DESCRIPTION" column |
| immunization_unwanted.csv  | Count of Unwanted Data on Dataframe         |

• **Inuenza seasonal injectable preservative free** was the most frequently occurring entry (see <i>immunization_uvc1.csv</i>).
<br><br>
• <i>immunization.csv</i> contains no unwanted data (see <i>immunization_unwanted.csv</i>)

#### 1.1.7) medications.csv

In [10]:
# MEDICATION #

df_temp = spark.read.csv("dataset_project_1/medications.csv", header=True)

# Unique Values Count
unique_writer(df_temp, "DESCRIPTION", "eda/understanding_entries/medications_uvc1.csv") # DESCRIPTION
unique_writer(df_temp, "REASONDESCRIPTION", "eda/understanding_entries/medications_uvc2.csv") # REASONDECRIPTION

count_unwanted("dataset_project_1/medications.csv", "eda/understanding_entries/medications_unwanted.csv")

# Cross Tabulation Operation
df_temp.crosstab("DESCRIPTION", "REASONDESCRIPTION").toPandas().to_csv("eda/understanding_entries/medications_ct1.csv", index=False)

# Unique Values Count on Dataframe filtered by null values
df_temp1 = df_temp.where(df_temp["REASONDESCRIPTION"].isNull()) # filtering by NaN values in REASONDESCRIPTION column
unique_writer(df_temp1, "DESCRIPTION", "eda/understanding_entries/medications_nuvc1.csv") #unique values count on DESCRIPTION

| **File**                 | **Operation**                                                                                   |
|:-------------------------|:------------------------------------------------------------------------------------------------|
| medications_uvc1.csv     | Unique Values Count on "DESCRIPTION" column                                                     |
| medications_uvc2.csv     | Unique Values Count on "REASONDESCRIPTION" column                                               |
| medications_ct1.csv      | Cross Tabulation operation on "DESCRIPTION" and "REASONDESCRIPTION"                             |
| medications_nuvc1.csv    | Unique Values Count on "DESCRIPTION" on dataframe filtered by NaN values in "REASONDESCRIPTION" |
| medications_unwanted.csv | Count of Unwanted data in the dataframe                                                         |

• **Hydrochlorothiazide 25 MG Oral Tablet** is the most common medication (see <i>medication_uvc1.csv</i>).
<br><br>
• Most medications are prescribed for minor medical reasons i.e. "REASONDESCRIPTION" having a
**NaN value**, the second most common reason is **Hypertension** (see <i>medications_uvc2.csv</i>).
<br><br>
• In the absence on of any major medical reason i.e. "REASONDESCRIPTION" column has **NaN value**
the most frequent entry in the data base is **Nitroglycerin 0.4 MG/ACTUAT Mucosal Spray** (see
<i>medication_nuvc1.csv</i>)
<br><br>
• It was observed that "REASONCODE" contains some **NaN values**, some values in the "PAYER
COVERAGE" and "TOTAL COST" columns are also **0**, although it is possible for a patient to have no
health coverage, the possibility of "TOTAL COST" column having **0** might point towards the presence
is any bad data in the medication.csv.


#### 1.1.8) observations.csv

In [11]:
# OBSERVATION #

df_temp = spark.read.csv("dataset_project_1/observations.csv", header=True)

# Unique Value Counts
unique_writer(df_temp, "DESCRIPTION", "eda/understanding_entries/observation_DESCRIPTION.csv")

count_unwanted("dataset_project_1/observations.csv", "eda/understanding_entries/observations_unwanted.csv")

| **File**                  |**Operations**                               |
|:--------------------------|:--------------------------------------------|
| observations_uvc.csv      | Unique Values Count on "DESCRIPTION" column |
| observations_unwanted.csv | Count of Unwanted Data in the Dataframe     |

• **Pain severity - 0-10 verbal numeric rating \[SCORE\]- Reported** is the most commonly performed
procedure.
<br><br>
• Based on the nature of the data it is difficult to draw any meaningful conclusion, thus a further analysis
is required to draw meaningful insights.
<br><br>
• Based on an initial analysis it was observed that "ENCOUNTER" and "UNITS" column has some **NaN values** and some values in the "VALUE" column are also zero. Thus based on an initial analysis it can be stated that <i>observations.csv</i> does have some unwanted data.

#### 1.1.9) procedures.csv

In [12]:
# PROCEDURES #

df_temp = spark.read.csv("dataset_project_1/procedures.csv", header=True)

# Unique Values Count
unique_writer(df_temp, "DESCRIPTION", "eda/understanding_entries/procedures_uvc1.csv") # DESCRIPTION column
unique_writer(df_temp, "REASONDESCRIPTION", "eda/understanding_entries/procedures_uvc2.csv") # REASONDESCRIPTION column

count_unwanted("dataset_project_1/procedures.csv", "eda/understanding_entries/procedures_unwabted.csv") # Unwanted Data

# Unique Values Count on Dataframe filtered by Null Values
df_temp1 = df_temp.where(df_temp["REASONDESCRIPTION"].isNull()) # Filtering by null values in REASONDESCRIPTION
unique_writer(df_temp1, "DESCRIPTION", "eda/understanding_entries/procedures_nuvc1.csv") # counting unique values in DESCRIPTION column

# Cross Tabulation Operation
df_temp.crosstab("DESCRIPTION", "REASONDESCRIPTION").toPandas().to_csv("eda/understanding_entries/procedures_ct1.csv", index=False) # DESCRIPTION and REASONDESCRIPTION

| **File**                | **Operation**                                                                                   |
|:------------------------|:------------------------------------------------------------------------------------------------|
| procedures_uvc1.csv     | Unique Values Count on "DESCRIPTION" column                                                     |
| procedures_uvc2.csv     | Unique Values Count on "REASONDESCRIPTION" column                                               |
| procedures_ct1.csv      | Cross Tabulation operation on "DESCRIPTION" and "REASONDESCRIPTION"                             |
| procedures_nuvc11.csv   | Unique Values Count on "DESCRIPTION" on dataframe filtered by NaN Values in "REASONDESCRIPTION" |
| procedures_unwanted.csv | Count of unwanted data on dataframe                                                             |


• **Medication Reconciliation (procedure)** is the most frequently performed procedure (see <i>procedures_uvc1.csv</i>).
<br><br>
• Most of the procedures are performed due to **Normal Pregnenecy** (see <i>procedures_uvc2.csv</i>)
<br><br>
• It was observed that "REASONDESCRIPTION" does contain some **NaN Values** (see <i>proceduresnuvc1.csv</i>).
However these cannot be regarded as missing values as this represents the cases where there is no major
medical reason.
<br><br>
• Even in the absence of any major medical reason it was observed that **Medication Reconciliation
(procedure)** was most frequently performed procedure(see <i>procedures_nuvc1.csv</i>) however it must be
observed that in the absence of any major medical reason i.e. "REASONDESCRIPTION" column has
a NaN Value.
<br><br>
• On a closer obervation it could be observed that a procedure could be performed for multiple medical
reasons (see <i>procedures_ct1.csv</i> )